In [1]:
# Cell 1: Install dependencies (run only once)
!pip install kafka-python requests


In [2]:
# Cell 2: Import libraries
import requests
import json
import time
from datetime import datetime
from kafka import KafkaProducer


In [3]:
# Cell 3: Setup Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=["kafka:29092"],  # เปลี่ยนให้ตรงกับ Kafka broker ของคุณ
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

print("✅ Kafka Producer Ready")


✅ Kafka Producer Ready


In [4]:
# Cell 4: Binance API fetch function
BINANCE_API = "https://fapi.binance.com/fapi/v1/fundingRate"

def fetch_funding_rate(symbol: str, limit: int = 1):
    """
    ดึงข้อมูล funding fee rate จาก Binance Futures
    symbol เช่น 'BTCUSDT'
    limit = จำนวน record (default 1 ล่าสุด)
    """
    params = {"symbol": symbol, "limit": limit}
    r = requests.get(BINANCE_API, params=params)
    r.raise_for_status()
    return r.json()


In [5]:
# Cell 5: Function ส่งข้อมูลเข้า Kafka
def stream(symbols):
    for symbol in symbols:
        data = fetch_funding_rate(symbol)
        for entry in data:
            message = {
                "exchange": "binance",
                "symbol": entry["symbol"],
                "funding_time": datetime.fromtimestamp(entry["fundingTime"]/1000).isoformat(),
                "funding_rate": float(entry["fundingRate"]),
                "mark_price": float(entry["markPrice"]) 
            }
            producer.send("funding_rate", message)
            print("📤 Sent:", message)


In [6]:
# Cell 6: Run streaming loop (Press Stop to interrupt)
symbols = ["BTCUSDT", "ETHUSDT", "BNBUSDT"]  # คุณสามารถโหลด symbol list จาก exchangeInfo API ได้

while True:
    stream(symbols)
    print("⏳ Sleeping 5 minutes...")
    time.sleep(300)  # 5 นาที


📤 Sent: {'exchange': 'binance', 'symbol': 'BTCUSDT', 'funding_time': '2025-08-23T16:00:00', 'funding_rate': 9.711e-05, 'mark_price': 114778.21648188}
📤 Sent: {'exchange': 'binance', 'symbol': 'ETHUSDT', 'funding_time': '2025-08-23T16:00:00', 'funding_rate': 0.0001, 'mark_price': 4749.5566938}
📤 Sent: {'exchange': 'binance', 'symbol': 'BNBUSDT', 'funding_time': '2025-08-23T16:00:00', 'funding_rate': 0.00012106, 'mark_price': 889.17}
⏳ Sleeping 5 minutes...


KeyboardInterrupt: 